In [54]:
from bs4 import BeautifulSoup

import requests
import re
import regex
import json

I_master_sem1 = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_DATE=20.11.2017&ww_x_PLANMODELE=2211200510&ww_i_reportModel=2096516523&ww_x_GPS=2140957416&ww_i_reportModelXsl=2096516548&ww_x_HIVERETE=2936286'
C_master_sem1 = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_DATE=21.11.2017&ww_x_PLANMODELE=2211418626&ww_i_reportModel=2096516523&ww_x_GPS=2140962007&ww_i_reportModelXsl=2096516548&ww_x_HIVERETE=2936286'

days = ['LUNDI', 'MARDI', 'MERCREDI', 'JEUDI']


#change this to change json file output
JSON_OUTPUT_NAME = 'courses_scrapped.json'

In [2]:
r  = requests.get(I_master_sem1)



In [3]:
data = r.text


soup = BeautifulSoup(data)


print(soup.prettify())


<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="gedpublicreports.css?ww_x_path=Gestac.Moniteur.Style" rel="stylesheet" type="text/css"/>
 </head>
 <body alink="#666666" bgcolor="#ffffff" link="#666666" marginheight="0" marginwidth="5" vlink="#666666">
  <style type="text/css">
   tr.grisleger  {background-color: #f0f0f0}
  tr.white      {background-color: white}
  </style>
  <table width="100%">
   <tr>
    <td>
     <h1>
      Horaire par classe
     </h1>
    </td>
    <td>
     <h1>
      pour la semaine du 20.11.2017 au 27.11.2017
     </h1>
    </td>
   </tr>
   <tr>
    <td colspan="2">
     <h1>
      Informatique, 2017-2018, Master semestre 1
     </h1>
    </td>
   </tr>
  </table>
  <table border="0" width="100%">
   <tr>
    <th>
     Horaires
    </th>
    <th>
     Salles
    </th>
    <th>
    </th>
    <th>
    </th>
    <th>
     Matières
    </th>
    <th>
     Engagements
    </th>
   </tr>
   <tr>
    <td colspan="6

/Users/jon/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/jon/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [4]:
print(soup.title)
print(soup.name)

None
[document]


In [5]:
table = soup.find_all('table')[1]
t = []

row_marker = 0
for row in table.find_all('tr'):
    inner = []
    columns = row.find_all('td')
    for column in columns:
        txt = column.get_text()
        if txt != '' and txt[0:1]:
            inner.append(txt)
    if (inner !=[''] and inner !=[]):
        t.append(inner[0:5])

for entries in t:
    print(entries)

['LUNDI']
['08:15-10:00', 'MAA330', 'C', 'OPT', 'Set theory']
['Enseignant-e-(s): Duparc Jacques']
['10:15-12:00', 'INR113', 'C', 'OPT', 'Decentralized systems engineering']
['Enseignant-e-(s): Ford Bryan Alexander']
['10:15-12:00', 'INF119', 'C', 'OPT', 'Modern digital communications: a hands-on approach']
['Enseignant-e-(s): Rimoldi Bixio']
['10:15-12:00', 'MAA330', 'E', 'OPT', 'Set theory']
['Enseignant-e-(s): Duparc Jacques']
['13:15-15:00', 'ELA1', 'C', 'OBL', 'Information theory and coding']
['Enseignant-e-(s): Telatar Emre']
['14:15-16:00', 'INF119', 'C', 'OPT', 'Digital 3D Geometry Processing']
['Enseignant-e-(s): Pauly Mark']
['14:15-17:00', 'INM203', 'C', 'OPT', 'Mathematical foundations of signal processing']
['Enseignant-e-(s): Kolundzija Mihailo, Parhizkar Reza, Scholefield Adam James']
['15:15-17:00', 'CM1', 'C', 'OBL', 'Distributed algorithms']
['Enseignant-e-(s): Guerraoui Rachid']
['17:15-18:00', 'BC01BC02BC03', 'E', 'OBL', 'Distributed algorithms']
['Enseignant-e-(s):

In [37]:
def extract_rooms(text):
    pattern = r'[A-Z]+ *[0-9]+'
    ret = re.findall(pattern, text)
    return ret

def get_day_of_week(text):
    if (text == 'LUNDI'):
        return 0
    if (text == 'MARDI'):
        return 1
    if (text == 'MERCREDI'):
        return 2
    if (text == 'JEUDI'):
        return 3
    if (text == 'VENDREDI'):
        return 4
    if (text == 'SAMEDI'):
        return 5
    if (text == 'DIMANCHE'):
        return 6
    else:
        return -1 

def get_time_slots(text):
    
    start_hour = int(text[0:2]) - 8
    finishing_hour = int(text[6:8]) - 8
    
    ts = []
    for x in range(start_hour, finishing_hour):
        #print(x)
        ts.append(x)
    return ts


def build_course_span(day, times, rooms):
    ret = []
    for time in times:
        new_entry = {'day': day, 'time': time, 'room': rooms, 'activity': 0}
        ret.append(new_entry)
    return ret

def parse_course_table(table):
    ret = {}
    dow = -1
    ts = []
    rooms = []
    spe = ''
    name = ''
    teachers = ['']
    for entries in table:
        if entries == [] or entries == ['']:
            continue
        dow_test = get_day_of_week(entries[0])
        if dow_test != -1:
            dow = dow_test
            continue
        if entries[0][0:1].isdigit():
            if (len(entries) <= 3):
                print('problematic entry discarded: ')
                print(entries)
            else:
                
                #here we have a regular course schedule
                ts = get_time_slots(entries[0])
                rooms = extract_rooms(entries[1])
                spe = entries[2]
                name = entries[4]
        if (entries[0][0:10] == 'Enseignant'):
            #teachers = parse_teachers(entries[0])
            #if we reach techer entry we are done with the course
            #print(name)
            ts_built = {'timeslots': build_course_span(dow, ts, rooms)}
            
            if name in ret:
                old_ts = ret[name]['timeslots']
                new_ts = old_ts + ts_built['timeslots']
                ret[name] = {'timeslots': new_ts}
            else:
                ret[name] = ts_built
            
    return ret


courses_no_credits = parse_course_table(t)
            

problematic entry discarded: 
['16:15-19:00', 'SHS : Introduction au projetVoir horaire dans section SHS']


In [38]:
print(json.dumps(courses_no_credits, sort_keys=True,
                  indent=4, separators=(',', ': ')))

{
    "Advanced multiprocessor architecture": {
        "timeslots": [
            {
                "activity": 0,
                "day": 1,
                "room": [
                    "BC03"
                ],
                "time": 2
            },
            {
                "activity": 0,
                "day": 1,
                "room": [
                    "BC03"
                ],
                "time": 3
            },
            {
                "activity": 0,
                "day": 3,
                "room": [
                    "INM201"
                ],
                "time": 2
            },
            {
                "activity": 0,
                "day": 3,
                "room": [
                    "INM201"
                ],
                "time": 3
            }
        ]
    },
    "Applied data analysis": {
        "timeslots": [
            {
                "activity": 0,
                "day": 3,
                "room": [
                    "S

In [39]:
second_schedule = 'https://ic.epfl.ch/computer-science/study-plan_master'

In [40]:
r  = requests.get(second_schedule)

data = r.text
soup = BeautifulSoup(data)
print(soup.prettify())

/Users/jon/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/jon/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


<!DOCTYPE html>
<!--[if lt IE 7]><html lang="en" class="no-js lt-ie10 lt-ie9 lt-ie8 lt-ie7"><![endif]-->
<!--[if IE 7]><html lang="en" class="no-js ie7 lt-ie10 lt-ie9 lt-ie8"><![endif]-->
<!--[if IE 8]><html lang="en" class="no-js ie8 lt-ie10 lt-ie9"><![endif]-->
<!--[if IE 9]><html lang="en" class="no-js ie9 lt-ie10"><![endif]-->
<!--[if !IE]><!-->
<html class="no-js" lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <!--<![endif]-->
 <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
 <head>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <script language="JavaScript" type="text/javascript">
   var jahiaMainServletPath="/cms";
				var jahiaContextPath ="";
				var jahiaCoreWebdavPath ="/webdav";
  </script>
  <title>
   Study plan  | IC
  </title>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <meta content="width=device-width, initial-scale=1.0, shrink-to-fit=no" data-header-ver

In [41]:

    
    
    
raw_courses = soup.find('div', id=135978)

print(raw_courses)

<div class=" htmlBox" id="135978" style="">
<div class="relative">
<div class="content" id="content"><h3>Informatique 2017-18
     
   <a href="http://isa.epfl.ch/pe/plan_etude_bama_cyclemaster_in_en.pdf" target="_blank"><img alt="télécharger le plan en pdf" border="0" src="//isa.epfl.ch/images/gestacplus/etu/plan_fiche/ic_pdf.gif" title="télécharger le plan en pdf"/></a></h3><h4>Master project</h4><div class="line-up"><div class="first-line"><div class="langue"></div><div class="cours-title"><div class="titre bold">Courses</div><div class="titre">Code</div></div><div class="section"><div class="titre bold"> </div><div class="titre">Programs</div></div><div class="enseignement"><div class="titre bold"> </div><div class="titre">Lecturers</div></div><div class="specialisation"><div class="titre bold"> </div><div class="titre">Specialisation</div></div><div class="bachlor"><div class="titre_bachlor bold">Master 1 </div><div class="titre"><div class="cep">l</div><div class="cep">e</div><di

In [42]:
courses = raw_courses.find_all('div', class_='line-down')
print(courses)

[<div class="line-down"><div class="line"><div class="langue"><div class="langue francais"><div class="diet_icon"> </div> </div>
       
     </div><div class="cours"><div class="cours-name"><a href="http://isa.epfl.ch/imoniteur_ISAP/!itffichecours.htm?ww_i_matiere=652061370&amp;ww_x_anneeAcad=2017-2018&amp;ww_i_section=249847&amp;ww_i_niveau=&amp;ww_c_langue=en" target="_blank">Internship credited with Master Project (master in Computer Science)</a> <i>(L'étudiant doit effectuer un stage de 8 semaines pendant l'été ou 6 mois après un semestre de Master.)</i></div><div class="cours-code">CS-595 </div><div class="section-name">IN </div><div class="enseignement-name">Profs divers
        
      </div></div><div class="specialisation">
        
      </div><div class=" bachlor local-color bachlor-color "><div class="bachlor-text"><div class="cep"> 
          </div><div class="cep"> 
          </div><div class="cep">320h 
          </div></div></div><div class=" bachlor local-color-light b

In [43]:
for c in courses:
    print(c.find('a', href=True)['href'])

http://isa.epfl.ch/imoniteur_ISAP/!itffichecours.htm?ww_i_matiere=652061370&ww_x_anneeAcad=2017-2018&ww_i_section=249847&ww_i_niveau=&ww_c_langue=en
http://isa.epfl.ch/imoniteur_ISAP/!itffichecours.htm?ww_i_matiere=128778875&ww_x_anneeAcad=2017-2018&ww_i_section=249847&ww_i_niveau=&ww_c_langue=en
http://isa.epfl.ch/imoniteur_ISAP/!itffichecours.htm?ww_i_matiere=1772055&ww_x_anneeAcad=2017-2018&ww_i_section=249847&ww_i_niveau=&ww_c_langue=en
http://isa.epfl.ch/imoniteur_ISAP/!itffichecours.htm?ww_i_matiere=1770973&ww_x_anneeAcad=2017-2018&ww_i_section=249847&ww_i_niveau=&ww_c_langue=en
http://isa.epfl.ch/imoniteur_ISAP/!itffichecours.htm?ww_i_matiere=1888826574&ww_x_anneeAcad=2017-2018&ww_i_section=249847&ww_i_niveau=&ww_c_langue=en
http://isa.epfl.ch/imoniteur_ISAP/!itffichecours.htm?ww_i_matiere=1773287&ww_x_anneeAcad=2017-2018&ww_i_section=249847&ww_i_niveau=&ww_c_langue=en
http://isa.epfl.ch/imoniteur_ISAP/!itffichecours.htm?ww_i_matiere=1773292&ww_x_anneeAcad=2017-2018&ww_i_section

In [44]:
test_url = 'http://isa.epfl.ch/imoniteur_ISAP/!itffichecours.htm?ww_i_matiere=1772055&ww_x_anneeAcad=2017-2018&ww_i_section=249847&ww_i_niveau=&ww_c_langue=en'

In [45]:
fos = 'http://isa.epfl.ch/imoniteur_ISAP/!itffichecours.htm?ww_i_matiere=71840397&ww_x_anneeAcad=2017-2018&ww_i_section=249847&ww_i_niveau=&ww_c_langue=en'

In [46]:
def parse_course_book(url):
    
    
    r  = requests.get(url)
    
    data = r.text
    
    
    soup = BeautifulSoup(data)
    
    #default values
    hours = 0
    name = ''
    summary = ''
    credits = 0
    
    #we already have it with previous function so we skip it for now
    timeslots = []
    
    if (soup.find(type='header', var="ITEMPLAN_XMATIERE_EN") != None):
        name = soup.find(type='header', var="ITEMPLAN_XMATIERE_EN").text.strip()        
        summary = soup.find(var='RUBRIQUE_RESUME').text.strip()
    
        credits = int(soup.find(type='header', var="ITEMPLAN_ECTS_OU_COEFF").text.strip())  
    
        
    
    
    return (name, summary, credits)

In [47]:
parse_course_book(fos)

/Users/jon/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/jon/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


('Foundations of software',
 'The course introduces the foundations on which programs and programming languages are built. It introduces syntax, types and semantics as building blocks that together define the properties of a program part or a language. Students will learn how to apply these concepts in their reasoning.',
 4)

In [48]:
print(courses_no_credits)

{'Set theory': {'timeslots': [{'day': 0, 'time': 0, 'room': ['MAA330'], 'activity': 0}, {'day': 0, 'time': 1, 'room': ['MAA330'], 'activity': 0}, {'day': 0, 'time': 2, 'room': ['MAA330'], 'activity': 0}, {'day': 0, 'time': 3, 'room': ['MAA330'], 'activity': 0}]}, 'Decentralized systems engineering': {'timeslots': [{'day': 0, 'time': 2, 'room': ['INR113'], 'activity': 0}, {'day': 0, 'time': 3, 'room': ['INR113'], 'activity': 0}, {'day': 4, 'time': 5, 'room': ['INF213'], 'activity': 0}, {'day': 4, 'time': 6, 'room': ['INF213'], 'activity': 0}, {'day': 4, 'time': 7, 'room': ['INF213'], 'activity': 0}, {'day': 4, 'time': 8, 'room': ['INF213'], 'activity': 0}]}, 'Modern digital communications: a hands-on approach': {'timeslots': [{'day': 0, 'time': 2, 'room': ['INF119'], 'activity': 0}, {'day': 0, 'time': 3, 'room': ['INF119'], 'activity': 0}, {'day': 2, 'time': 3, 'room': ['INF119'], 'activity': 0}, {'day': 2, 'time': 4, 'room': ['INF119'], 'activity': 0}]}, 'Information theory and coding'

In [49]:
def add_infos_to_courses(raw_courses, course_table):
    
    courses_url = raw_courses.find_all('div', class_='line-down')
    for c in courses_url:
        url = c.find('a', href=True)['href']
        (name, summary, credits) = parse_course_book(url)
        if not name in course_table:
            continue
        course_table[name]['credits'] = credits
        course_table[name]['summary'] = summary
        
    return course_table

In [50]:
courses_with_credits = add_infos_to_courses(raw_courses, courses_no_credits)

/Users/jon/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/jon/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [56]:
json_output = open(JSON_OUTPUT_NAME, 'w')
json_output.write(json.dumps(courses_with_credits, sort_keys=True,
                  indent=4, separators=(',', ': ')))
json_output.close()

In [53]:
print(json.dumps(courses_with_credits, sort_keys=True,
                  indent=4, separators=(',', ': ')))

{
    "Advanced multiprocessor architecture": {
        "credits": 6,
        "summary": "Multiprocessors are now the defacto building blocks for all computer systems. This course will build upon the basic concepts offered in Computer Architecture I to cover the architecture and organization of modern multiprocessors from mobile and embedded platforms to servers, data centers and cloud computing platforms.",
        "timeslots": [
            {
                "activity": 0,
                "day": 1,
                "room": [
                    "BC03"
                ],
                "time": 2
            },
            {
                "activity": 0,
                "day": 1,
                "room": [
                    "BC03"
                ],
                "time": 3
            },
            {
                "activity": 0,
                "day": 3,
                "room": [
                    "INM201"
                ],
                "time": 2
            },
           